In [2]:
# Dependencies
import requests
import json
from pprint import pprint
from config import client_id, client_secret

Following instructions from: https://stmorse.github.io/journal/spotify-api.html
AND from Spotify documentation:
https://developer.spotify.com/documentation/general/guides/authorization-guide/#list-of-scopes

## Authorization flow

Step 1: Request authorization to access data. No code. 
Example Grant access URL:
https://accounts.spotify.com/authorize?response_type=code&client_id=5d67ef47a63a42f08feb4c824ee174f7&scope=user-library-read&redirect_uri=https%3A%2F%2Fgoogle.com%2Fcallback&show_dialog=true

Once accept, data can now be requested.

Note: if changing `scope`, this step and step 2 need to be repeated.
Scope for the following:
- User's Top Artists and Tracks: `user-top-read`
- List of a User's Playlists: `playlist-read-private`

In [3]:
# Once access is granted, copy the code from the URL
code = 'AQDHk5w01qWgoh8ClHrMHg9arLydL0zy6Zs2l3bl3RP0kEYfdHViimQ1A0aWOwrA30WuMlwoW39eUCPkjhaUwNvjZwwMOtIknkpUnf58oXQxg4twZ-xAahJfdCIqKDTbUVwsdxD4PTprqQ3Jdnv-iLlEgPrhQhRMucCOWVpCE2aRSVOJDWtwuF69ZebIXw'

Step 2: Request to get refresh and access token
- Make POST request to Spotify on /api/token endpoint
- Require parameters specfied accordingly.

In [4]:
# POST - REQUEST BODY PARAMETER, skipping header parameter using alternative way
request_body_parameter = {
    'grant_type': 'authorization_code',
    'code': code,
    'redirect_uri': 'https://google.com/callback',
    'client_id': client_id,
    'client_secret': client_secret,
}

In [5]:
# Pass an access token - POST a request with client credentials and save part of response
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST - REQUEST BODY PARAMETER
auth_response = requests.post(AUTH_URL, request_body_parameter)

# convert the response to JSON
auth_response_data = auth_response.json()

# Try print it out to get refresh_token
print(auth_response_data)

{'access_token': 'BQDiFXFPKMyXpLcZJAab7crMEaGXRRA4eOj4iyDcpKaTudLEFIiBI47sfgPik6GRw9pe5jjbhQJcCKhzAhl7BvWVKGf_Tjz-S502zsVis-CXDJGU7-_E7209s_Iuo-aHLUZLPymq3WmJ22J0oiQ', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQA1YePrNTTY_I-kOBiWb-fk9Wm6F-mOjvfSTm25jjVvfgqtkkQVs-HESP_zRW802itP0DKSqIOhTA43yj9Y66fpM86y6nmfaAgqFKoq31kc7p5ftqF8Md6PiVJcmTQD-To', 'scope': 'user-top-read'}


In [6]:
# Save the access token
access_token = auth_response_data['access_token']

# Save the refresh token
refresh_token = auth_response_data['refresh_token']

Step 3: Use access token to access data of interests(?)
Note: when access token expires, it can't be used anymore.

#### Get a User's Top Artists and Tracks

In [19]:
# Form GET request to API server with access_token in the header
headers = {
    'Authorization': f'Bearer {access_token}'
}

# base URL of all Spotify API endpoints
base_url = 'https://api.spotify.com/v1/'

# user's top track
top_track = 'me/top/tracks'
time_range = 'time_range=medium_term'
limit = 'limit=50'
# offset = 

In [20]:
track_json = requests.get(base_url + top_track + '?' + time_range + '&' + limit
                          , headers=headers).json()

In [25]:
# Save outout to text file to go parse later...
with open('top_tracks.json', 'w') as outfile:
    json.dump(track_json, outfile, indent=2)

Step 4: When access token expires, refresh token to the rescue (from step 2)

In [ ]:
# Base 64 encoded string that contains the client ID and client secret key.
import base64

def encode_base64(message):
    message_bytes = message.encode('ascii')
    base64_bytes = base64.b64encode(message_bytes)
    base64_message = base64_bytes.decode('ascii') 
    return base64_message
#     return str(base64_bytes, "utf-8")
    
print(encode_base64(f'{client_id}:{client_secret}'))

In [ ]:
# Get the new refresh access token and update - only when request fails 
def refresh_access_token():   
    # build body parameter
    body = {
        'grant_type': 'refresh_token',
        'refresh_token': refresh_token
    }
    
    # base64 string from above encode_base64 function
    auth_base64 = encode_base64(f'{client_id}:{client_secret}')
    
    # build header parameter
    header = {
        'Authorization': f'Basic {auth_base64}'
    }
    
    request_access_token = requests.post(AUTH_URL, body, headers=header)
    request_access_token_data = request_access_token.json()
    # Update access_token (for step 3)
    access_token = request_access_token_data['access_token']
    # Update headers (for step 3)
    headers = {
    'Authorization': f'Bearer {access_token}'
    }
    
refresh_access_token()